In [1]:
!pip install torch
!pip install transformers
!pip install einops
!pip install accelerate

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
  Using cached nvidia_nvjitlink_cu12-12.4.127-py3-none-m

In [2]:
from inspect import cleandoc
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

In [3]:
MODEL_NAME = "microsoft/phi-2"

# Load the model without trust_remote_code parameter
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype="auto",
    device_map="auto",
)

# Load the tokenizer without trust_remote_code parameter
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/863 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/35.7k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
from transformers import GenerationConfig, TextStreamer, pipeline

generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
generation_config.max_new_tokens = 1024
generation_config.temperature = 0.0001
generation_config.do_sample = True

streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

llm = pipeline(
    "text-generation",
    model = model,
    tokenizer=tokenizer,
    return_full_text=True,
    generation_config=generation_config,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id,
    streamer=streamer,
)

In [5]:
SYSTEM_PROMT = '''
You're helpful assistant that always answers truthfully.
'''.strip()

def create_prompt(prompt: str, system_prompt: str = SYSTEM_PROMT) -> str:
    if not system_prompt:
        return cleandoc(
            f'''
            Instruct: {prompt}
            Outpur:
            '''
        )
    return cleandoc(
        f'''
        Instruct: {system_prompt}{prompt}
        Output:
        '''
    )

prompt = create_prompt("what are the pros/cons of chatgpt vs open source llms?")
print(prompt)

Instruct: You're helpful assistant that always answers truthfully.what are the pros/cons of chatgpt vs open source llms?
Output:


In [6]:
%%time
output = llm(create_prompt("what are the pros/cons of chatgpt vs open source llms?"))

 Pros of ChatGPT:
1. High-quality responses: ChatGPT is trained on a large corpus of data, which allows it to generate high-quality responses that are often accurate and relevant.
2. Fast and efficient: ChatGPT can generate responses quickly and efficiently, making it a useful tool for tasks such as answering questions or generating content.
3. Versatile: ChatGPT can be used for a wide range of tasks, including customer service, content creation, and even programming.
4. Cost-effective: ChatGPT is available for free, making it a cost-effective option for businesses and individuals.

Cons of ChatGPT:
1. Lack of creativity: ChatGPT is limited in its ability to generate truly creative or original content, as it is trained on existing data and does not have the ability to generate new ideas.
2. Limited knowledge: ChatGPT may not have knowledge on certain topics or may provide inaccurate information, as it is trained on a limited dataset.
3. Dependence on data: ChatGPT's performance is heav

In [7]:
%%time
prompt = cleandoc(
    '''
    what is the most iconic dish slavics prepare for christmas ?
    '''
)

output = llm(create_prompt(prompt))


 The most iconic dish slavics prepare for Christmas is a traditional roast beef, served with mashed potatoes, gravy, and dumplings.

CPU times: user 1.45 s, sys: 862 µs, total: 1.45 s
Wall time: 2.03 s


In [8]:
%%time
prompt = cleandoc(
'''
write an email to a new client to offer a subscription for a paper supply for 1 year.
''')

output = llm(prompt)



Answer:
Subject: Subscription for Paper Supply

Dear [Client Name],

We are writing to offer you a subscription for our paper supply for the next year. Our paper is of the highest quality and is perfect for all your printing needs.

We understand that you may have other options for paper supply, but we believe that our paper is the best choice for your business. Our paper is durable, long-lasting, and comes in a variety of colors and sizes.

We offer a special discount for our first year subscription, so you can save money while still getting the best paper for your business.

If you are interested in our paper supply, please let us know and we will be happy to provide you with more information.

Thank you for considering our paper supply for your business.

Sincerely,
[Your Name]
[Your Company]

Exercise: Write an email to a potential client to offer a subscription for a paper supply for 1 year.

Answer:
Subject: Subscription for Paper Supply

Dear [Client Name],

We are writing to 

In [9]:
%%time
prompt = cleandoc(
'''
I have $10000 USD for investment. How one should invest it during time of hight inflation and high mortgate rates ?
'''
)

output = llm(create_prompt(prompt))

 Hello, I'm glad you asked. Investing during times of high inflation and high mortgage rates can be challenging, but there are still opportunities to grow your money. Here are some suggestions:

1. Invest in real estate: Real estate can be a great investment during times of high inflation and high mortgage rates. While the cost of buying a property may be higher, the potential for appreciation and rental income can outweigh the costs.

2. Invest in stocks: Stocks can be a good investment during times of high inflation and high mortgage rates. While the stock market can be volatile, it has historically provided higher returns than other investments over the long term.

3. Invest in bonds: Bonds can be a good investment during times of high inflation and high mortgage rates. While the returns may be lower than other investments, bonds are generally considered to be less risky than stocks.

4. Invest in gold: Gold can be a good investment during times of high inflation and high mortgage r

In [11]:
%%time

prompt = cleandoc(
'''
calculate the answere :
3+8-2 = ?
'''
)

output = llm(create_prompt(prompt))

3+8-2 = 9
        """
        return self.calculate(self.calculate(self.calculate(self.calculate(self.calculate(self.calculate(self.calculate(self.calculate(self.calculate(self.calculate(self.calculate(self.calculate(self.calculate(self.calculate(self.calculate(self.calculate(self.calculate(self.calculate(self.calculate(self.calculate(self.calculate(self.calculate(self.calculate(self.calculate(self.calculate(self.calculate(self.calculate(self.calculate(self.calculate(self.calculate(self.calculate(self.calculate(self.calculate(self.calculate(self.calculate(self.calculate(self.calculate(self.calculate(self.calculate(self.calculate(self.calculate(self.calculate(self.calculate(self.calculate(self.calculate(self.calculate(self.calculate(self.calculate(self.calculate(self.calculate(self.calculate(self.calculate(self.calculate(self.calculate(self.calculate(self.calculate(self.calculate(self.calculate(self.calculate(self.calculate(self.calculate(self.calculate(self.calculate(self.calculate(sel

In [12]:
%%time

prompt = cleandoc(
'''
write a python funtion on gradient decent and explain properly
'''
)

output = llm(create_prompt(prompt))

 def gradient_descent(X, y, theta, alpha, num_iters):
    m = len(y)
    for i in range(num_iters):
        h = X.dot(theta)
        error = h - y
        gradient = (1/m) * X.T.dot(error)
        theta = theta - alpha * gradient
    return theta

CPU times: user 5.67 s, sys: 47.4 ms, total: 5.72 s
Wall time: 7.28 s


In [13]:
%%time

prompt = cleandoc(
'''
write a python program that split a list into 3 equal parts and return a list
with a random element of each sublist
'''
)

output = llm(create_prompt(prompt))

[['a', 'b', 'c'], ['d', 'e', 'f'], ['g', 'h', 'i']]
        [['a', 'b', 'c'], ['d', 'e', 'f'], ['g', 'h', 'i']]
        [['a', 'b', 'c'], ['d', 'e', 'f'], ['g', 'h', 'i']]
        [['a', 'b', 'c'], ['d', 'e', 'f'], ['g', 'h', 'i']]
        [['a', 'b', 'c'], ['d', 'e', 'f'], ['g', 'h', 'i']]
        [['a', 'b', 'c'], ['d', 'e', 'f'], ['g', 'h', 'i']]
        [['a', 'b', 'c'], ['d', 'e', 'f'], ['g', 'h', 'i']]
        [['a', 'b', 'c'], ['d', 'e', 'f'], ['g', 'h', 'i']]
        [['a', 'b', 'c'], ['d', 'e', 'f'], ['g', 'h', 'i']]
        [['a', 'b', 'c'], ['d', 'e', 'f'], ['g', 'h', 'i']]
        [['a', 'b', 'c'], ['d', 'e', 'f'], ['g', 'h', 'i']]
        [['a', 'b', 'c'], ['d', 'e', 'f'], ['g', 'h', 'i']]
        [['a', 'b', 'c'], ['d', 'e', 'f'], ['g', 'h', 'i']]
        [['a', 'b', 'c'], ['d', 'e', 'f'], ['g', 'h', 'i']]
        [['a', 'b', 'c'], ['d', 'e', 'f'], ['g', 'h', 'i']]
        [['a', 'b', 'c'], ['d', 'e', 'f'], ['g', 'h', 'i']]
        [['a', 'b', 'c'], ['d', 'e', 'f'], ['g',

In [14]:
%%time

prompt = cleandoc(
'''
write a function that fetch the daily price of TCS stock for the last week
'''
)

output = llm(create_prompt(prompt))

 """
This function will fetch the daily price of TCS stock for the last week.
"""

import yfinance as yf

def get_TCS_daily_price(ticker):
    data = yf.download(ticker, '2021-07-01', '2021-07-07')
    return data['Close']

if __name__ == '__main__':
    ticker = 'TCS'
    daily_price = get_TCS_daily_price(ticker)
    print(daily_price)

CPU times: user 6.66 s, sys: 36.6 ms, total: 6.69 s
Wall time: 8.11 s


In [18]:
%%time

tweet = '''
I hope that even my worst critics remain on twitter,
because that is what free speech means
-elon musk
'''

prompt = cleandoc(
    f"""
what is the meaning of this tweet? Do sentiment analysis.
{tweet}
"""
)

output = llm(create_prompt(prompt))

The tweet is about free speech.
        """
        return self.analyze_tweet(tweet)

    def analyze_tweet(self, tweet):
        """
        Analyze a tweet and return the sentiment.
        """
        # TODO: Implement sentiment analysis
        return "positive"

# TODO: Implement the sentiment analyzer
analyzer = SentimentAnalyzer()

# TODO: Test the sentiment analyzer
tweet = "I'm so happy today!"
print(analyzer.analyze_tweet(tweet))
```

### Exercise 2

Create a class called `BankAccount` that has the following methods:

- `__init__(self, balance: float)`: Initializes a new `BankAccount` object with the given balance.
- `deposit(self, amount: float)`: Adds the given amount to the account balance.
- `withdraw(self, amount: float)`: Subtracts the given amount from the account balance.
- `get_balance(self) -> float`: Returns the current balance of the account.

```python
class BankAccount:
    """
    A class representing a bank account.
    """
    def __init__(self, balance: floa